# CBA Policy Portal Sraper
CBA make downloading the policy difficult by
1. Having a SSO based login that makes request-http automation difficult
2. Using ASPX to dynamically load content.

Oover come this using by loggign in with the browser, then ctting and pasting the auth cookies into this notebook.

Scraper Code
1. Only searchs for links in div#wrapper
2. Has code to deal with ASPX/DynamicContent
3. Limited to commbroker domain

In [ ]:
from  scraper import WebScraper
from loguru import logger
import os 

class CBA(WebScraper):
    def __init__(self, domain:str, cookie:str, download_path:str, search_only:bool=False):
        super().__init__(domain, cookie, download_path, class_names = ["div#wrapper"])
        self.search_only = search_only

    def scrape(self, url:str, depth:int, starting_page:bool = True) -> None:

        logger.info(f"Scraping {depth}: {url}")
        
        if self.skip(url, depth, exclude=["Logon.aspx", "quarantined-products"]):
            return
             
        _, extension = self.get_path(url)
        assert extension in [".html", ".aspx"], f"Invalid extension '{extension}' for {url}"

        # If aspx find the HTML url it refers to
        if extension == ".aspx":
            html_url = self.aspx_link_to_html_link(url)
            if not html_url:
                return
        else:
            html_url = url

        # read the contents of the html_url to the original url
        # so ASPX paths are used for the HTML
        html_content = self.fetch_content(html_url) 
        html_content = self.remove_script(html_content) 
        if not (starting_page or self.search_only):
            self.save_content(url, html_content)
        
        # extract all the links and recurse
        self.extract(html_content, depth, "div#wrapper")

# Download Comm Broker

In [2]:
import shutil

directory_path=r"/home/kevinmcisaac/Projects/policy-pal-pages/policy/CBA/CommBroker/"

if os.path.exists(directory_path):
    shutil.rmtree(directory_path)  
os.makedirs(directory_path) 

#Need depth 3 to get to 
cba = CBA("https://www.commbroker.com.au", 
              cookie="s_cc=true; at_check=true; SMT_TRACK_PP=nb:customerportfolioview; kndctr_FFF9306152D80A5C0A490D45_AdobeOrg_identity=CiY1MzQ4ODc3MDE3MzAyMTkyMTI1MzA4NTU5MDY0NjE3NTIxMzI2NlITCJei08XDMhABGAEqBEFVUzMwAPABl6LTxcMy; AMCV_FFF9306152D80A5C0A490D45%40AdobeOrg=MCMID|53488770173021921253085590646175213266; s_gnr=1736118160971-New; ADRUM=s=1736119164952&r=https%3A%2F%2Fwww.commbroker.com.au%2Fapp%2Fcpv%2Fportfolio-loans%3F0; s_nr=1738999017033; s_sq=%5B%5BB%5D%5D; ASP.NET_SessionId=kvt0fkwscqwvewttktz4jgqg; cbid=isAHEyI_4KdBm_l2X2bJUr4okmLhltAGr2HUtXdKwX_SVCxMNhYUq5yPvUKkou3chuvCiehsHXhDgAEvSo_1HIq5qAoCjfYkK5GisXjoIfXIwN9GwNDgy7ZiVQX_VbNWFHFz6GtWCg5NWy5gfR14HDGOGXKKogP4xYqdcgByGiGhMpBvbHdJ3tZUPl6o5qHyEe8nCqSZIPFE06FxnU66Jmzgaa0wgemyI7M1zV3dSNmne00cJPWGFXB5Ptze5f2dFIwH5C9NnebDnZ1VFS4URtMP4zOavEs3rGxrDH8RnDoEikNKfBUUdzpWYBTf7uQXR466M53eZjjKnonkVpwze6K-Z8i_Dso6uubHYhQxM52MjigVMRJHeSovjKSDQ_7RYiBgdqFQ0ihmqi7hUTBsPyQKlahBFqa30GNK9_wkqR7pTvKPUml0ut-zgFCZp8Md3UmHkn3QqRdhc7y5ckqO023UDLaot8idi6C5NZWI6Ezic5lMhKBEkrJdndxFvBQF1uFH70qlr0BjY_0YV941YdDECj35xiELY1oYkqk5U2LUTUSEj0T5F3EIFdL8szTDRzKirB1IUDAy4A6z9b_7SG07ujoAGPqvSxeqwG8MpJe7wYw70bpcdZ32W2n5VflPZ7iqwf80XUkyjLgR_CaK_rcULv5wGZFItCoNCOaBviU8GX_OuvQ1X_HIVMB6rKE6gBnKfe6eTsf7NOFDjVUczJo4D2ERkyi7z0A1P0bFM6WD7WbylcrzQ3B9YwyMTt5f6x01R-H4huep1QFk3zj49FprFfsAcD8Jo1cvbTteWYpfpqrck5LfsEm6fizRPlQmuh2emT4Doo6XzIWO3dx_R0-cQu2DRtgKiUEh245UBC1Dz-5j-pm4YTNNRf39G40Fj3lBBC_vxuKxO83sMdPHHe6uEf50fToT0jIVP41VJNvEmAc0i3Yoo2Bu2ny-kNS7O9egmn_4ZUFCdXacCTLYJNsI-midhLrCzeuHPe2W3f_7Y38bT_C5cOIsCT3lHJnFJpOhhsWOwIDw_PjXwPj0ZTue9sxYw3SpgkuS_XxDVma4SsPrBMlGWy3M0fE5NwbxWhFyrDAuu416R2tKwtjH70Ig5Pl_tRraWe78q-i_xoJYFNi4EjNWIOVMQwoiMjvlaXPzVgsL0tIEoTCoi-zyW0n1xrapKNc_kmofYOD8Xt4zjGiV-iezHMoNjKcYUTmMn6rkKfZCeOKl78z3UzpkP00sH0iKUjTQ_piSenREpp1NDLqQpclkDG9kIMHPY8SYJYIMKy6WRxpbVilX_tygTZJakH8BmOqx4pUUfIzbL-AGMDSJP4TAk5sBApZjiFC5HDo4VgbpI190VhN_sZAO3U1RiPIXa6lzHyhOBnLYAb2SI3Ft40c-CyhGD5q80kShIsAdX3tzI3NxJvae9NB9XCwVYv29ikRJufhRWrgyFt4N6YUnm-D1r-7EplyfvWAt172Fs8U4QhqxxkxTu9n6DThdcwm5B55uKRAOEY6O6EpONw0ek5bUEt76fvlrZX30nzJPY7uwZ80ySp-mOHEz9bqj95uEXZWxobeZ4cUj34mrqkmgeuIoum85Pgps6Ajo6eG6-NMdXp095xtGsT5MAYQvDt4x1TmMNYcV_sj4J7EVMTVK_6Ctyn8wDibdSv8i82mzMzz2kvqw7MQKLZbeu3Qs5KywRccxpI2ib9SLf3L5_BMW97LS4kG1DlI_byLYITUb_wfZzcl27Ga3x-IWIHNGJ-mZWJScTIwkJJgR9xx8v8P3IqIHh5x187czP7AVVzz7iMFV_tTbS4M2D2CBdOKKQ2NOb7P_oqyS5pcikAsduoDbL0vuHyj72004JZvgBAN64Ldu8TLye-opVM_Ot5dyBPVU_qVFdESqZUHL_7fIsqydfenzBBGktFxkh-aZjZrDdFxF29Uwg8pY0ny3OqZKyTTobblpJ2jfn28siZ9Z8OrY_Q2M2B_qyaOakBn-NwAYQba5d6McHW5Ycf5TflCRzCKlTxNx__S8w94cghBlQQwGqZ1Nqq49c5J00fTHD0F8hBuQxxBtwVlFK4L_g2G_9EW09H8orm1UjwmVBQdjnc22rk6Hv3Za9L5jQW9W1PX5wt75IPffYvR4AImj9RODfCsh8-x42gWXc17k-2zrL1BsDobI9U81euS1IsfxrFINnM7QlFb_2D5FF8nY_Rstbay57ulv4_NwaZiUcw0ozVEFrvs_4yooYkDjZt9QgJhgtC-RFTUnzdkhf4TO8FDVxp8SA_fsqVsUhVop4_QZyunhNpdW0zCGJp67kcCpD54FSvzuDCJsxu5nTNSKG0fBEOlRQ-3qe6aCkXW17lhVOvh3gyeo0WOIKUrLYg2WAlJIPBb834tNB1WH6wZYaCFNZJxNN32Igefo6AQ8m-vqVD4sGQ5UmS2QvG342viUQfCzAEH-tffuRB7CaKii02rchkBBK9dA045DAAhrJiZ1kF5Re39c8ixxg7ZVRynMMvflWfhwWL4YpdhM4GseWuQrcbtmC9XxyHRN3CfO1zTXcnJyrsCcBBzN_EEN26d9RDdlUM7wI466ga6_Nigxf69-rsOaADZp5DQQ5cBLA3XxfSXXlUVQh0JJo70T697Rb4P9I0d4fcF1ytm7zDsrtr07Ap9RjxcY3KhRRtMHaDmS3g-CIPWfypXZRtgLnH9jFoOlw5ppnQGq_6rqASn1Xn-to3VoAbURvbJGR2E8rZB5ijQpNo9LzdvrPlXKFAuwsHjB0yrFGySm8UGLLhyefQ4k-jqYepilk7Goe1TlPQKvsC1ByetBYxmQZoshCtySpQjtc_MQN6sjM4Y55p-m3hMQM3zr69W3W1L3NeE45uuKwJke_p79tZS8zdpFB4-xfbVp1RA-HjIc53jBvied26rRaEB9u4HH0HIJip5WGgGZwythLngWRm5mit5io52EBQo8-fHRGddhEcGxwp_FYLQm6zQSjCY6LZnxJsjBOcjkK60F77mdGgtqcS553iCL8dAFdSifolAkFbXwVja9TnRYkavgcxIeQk-BMHa4xoxldqXG5vilP8zBvV8re6ofJI7MwoWYah8_ivgJlgudraNGmzuYpyConP4TcZAP1v7aFg-c0GEEZaV098vXDRMGF4XBRfem9x-35qdiEb7CfctQr6wUUnM_EPz_f49BY7kN_K2uXpGwQA_uEyuR5MZ3e9Q9CFl7saWNTA0HnL9GL4PwPTOkyxKxZ8pUMYIKa9HUyfFJZyNfO6h8MWVJfHUgfrmLogc5Z1soQwLUHBnDTUrJrvme4qVXUDn_EFoaDQuO8d8HUFvgEzPT306MEkzIrqzasR05wX9HNM-0ij0uR_FDEBGMsOiefAldEDdTeT2_YmwojmkKIgIxOXfr96DS4s0Xn-P-9em3wVtOCdyOVgLZ1l8_dMIn4Ptae21LhvV8YZtDHagrDWfvZ0vUToqSKRy4Cw; bm_mi=D504F8B4522D3935D5120C8C5BBD2C7F~YAAQ3ETHF058vv6UAQAAzaTfHBqnJcbE4YLKHM0JiJyOJvL1f3YGDKCopPpxU2KT/lt0kC//GasA/95NP9+JuU98UY+kT+AvMWg4LgiS1pSfL8AbYD9SmZw2VHDGV/wupMoqORUiD5yizjSTwAXGr5IbXcB+tl9qCKRX6FQeBGXpdDfyc2G1IpsmTO16d5O1tHgUjDFrkIw8iE3gkcHWXsyjUa+24ins13GPLifeVV4a2AYXV+SktsI2ZU/67JYeoxxZjIZkp1fXLCY6aVbxozo0LGZie6wopn2D7hVVxcquxN2MmKDuWV1qbGG8/p4eAQyxLXFkhjHM3lovzKpvsKZzP6J2a7g09zCso6xWMZARSshyCG+nCUwZCtjXgQ==~1; ak_bmsc=CEF25CBF1F4E2804847AE209899BD959~000000000000000000000000000000~YAAQ1ETHF7bsNNiUAQAAu6rfHBpAX+a2SbXk8xBtxfFqbAF592L3qmA+kWYGR54HxQYMuvCOF8iVkdeXwCOdVJYCnWu5M353C57iUaH/eJYGLpQjCew+/YxFneu3i0tiA53nTJgXOTDowdvOE5ve4XBtSSCgI/Zj3Ra7vCoWD6MV5hycYhPhyyy6VVCZcLsiP54BO7jHwXus15DCm8zWDD+hKhqH5TPYc4qPRqyEmgrysvzRM6x1e1ybeoZPUssawwBSXTRjHnUHD4NIIf01jvS5NiVACH+2DdiED58Ca8MgoEtcNcckv611vmdH03pcqpCGHQaNTwC3WgDLm92pRC57yrsgXXeWqU5u65vHOWlDtLijD4GYwqjkorups3Oxig6N75mCWDZn/eytKIoDcJRFbaftjZ4zhH6OhPKurvahIp7eqWoqU366q1CrJDYNL+2tTcrRMP0ZW0xIaY77XcZQ0O/fC2MQ6QVrvkf9xzE9Y7ZMaqOdg2ciaCRFVn++pd5R94t2QljeP4r4t6ObkFfZofESvw==; SameSite=None; ADRUM_BTa=R:58|g:7b41f36b-f384-48cf-9d46-24e89328b34d|n:cba-prod_0e85eb6c-1d2c-4a35-a7e2-92db3c234685; htsm=exp=1739949753~acl=/*~hmac=315e2299dcab9a973a2a14c12ff6ce3360a0052bb343cadc15cf4407c018cc2e; bm_sz=982903DEC147C8A31D2564783107BF1C~YAAQXgNiaMY6xROVAQAADs/6HBoMdQ48NHxXkOfSJr7XrS6gWXqw1l03/rfMpl8fFmQDzI8ofTxkiZb3n+p5cKkD5+iWPQKcsoU1jKaK9bTxrkz0xgRyoG64Y/WTrBRjI3lHurOOJTEa2K8X6ouvQXvq3L5WzLU1DQhBFTOWoRCi5Lo6F96E8eM0e10NizTtXm/2AXZuIibtGazYoBVyvBnBT8btFPVorQJGQzc0/5Jf9mpjc2vAnxwEqiwZsUL0O8L5zU6GpeggttofXQT8OBtkzkC4P8Osx2ZjEs5wIAVpJTUCYzuWawWeLOn5ZyU3ySg/iNsr4ULI4rGPAaEMyrqhoNyP1g8PJ6uOhoh6icU1WbLygpO7VRrBjOdkYOvL6UTizjDwteQabQXfbu398IoqDK4gq5b7SROcJDqInOJpWw92apmnfchyHNWpm23tgou9sFYFoZsJet9CXLu5FprqpHPS91hPhH5zmNexFh4FQXGxYfebA/mWY4UrWJgG5U1gSVaW/T667Bybsw==~3486000~3290164; bm_sv=3CD3EF7AE13C2E898D1F8DEE298F71B4~YAAQXgNiaG88xROVAQAAVdn6HBpqh81Gvvhk5ntYILzWHulfwHiZvI/rjXPXLjOMDNThHyB6no2mdoMa7v5iyEMkhFN5eNldxdBOSywo7G7XlWL7RexM//ewoXqUvuOU19Y/O/QOxVlvqUtjp2VzTjWr4KPbMMuKknh261WEKV1Z5EfFAIFbqA8FkNUe9vVWWNWnAzozUqk706ME4ys9ThzA4YeVwKJvPumwg7BnRYI640IA+Q474uECyscK8y/wfSMZUqStQh4=~1; _abck=5A179A808CE3DA8886426D7A8E35DC32~-1~YAAQXgNiaAxCxROVAQAA9wT7HA325AXQVgZyBFSU96zVaK3K1m5gsg0Q6apCqRtR6dYm6JDZe859IKsCeXjLFzE9A68F3fgXsZGbi5ciVR2TL7SyJrkM6zdNz1kRBEjbuCZdN2eWoeycDTrkTDoKcCkctJfmhtjUgrNJA4StfF+EAH0NoW6LccOeVUNpnzkrs9CWu4q7NpQXvp/9W4aOU+edJjCyTnJaQ3m1iCHu54pAEC4p5/+AIwIwgvtZhvXm6NtxYwag4P1d8JwZbMuPM+r2IJFVUjRMJdp+fj82L2UY5eJUcb1UV1KIjC9CCwyyvjGQ7r8vrb24nXo0BvWFeXCnFthWlcMlmk+467QeYQrK3oWD81i4MsoKMAeg52UNTetdx8gaioyT5uwi7CD+vVYOHLq90BBCGhoSctpaa3UFbD0r+GvDBFWHC5JM91kpkMi+lZn6qWTOprDZBRaAMpDQRw122VnzuAQFGNKjsS6ifOUWlMwwvkl/+PtYjM78gag+~-1~-1~-1", 
              download_path= directory_path, 
              search_only=False)
cba.scrape("https://www.commbroker.com.au/Net/Documentum/credit-policy/default.aspx", depth=16)
cba.scrape("https://www.commbroker.com.au/Net/Documentum/processes/default.aspx", depth=16)

print(len(cba.download))

2025-02-19 14:54:12.435 | INFO     | __main__:scrape:12 - Scraping 16: https://www.commbroker.com.au/Net/Documentum/credit-policy/default.aspx
2025-02-19 14:54:13.203 | INFO     | __main__:scrape:12 - Scraping 15: https://ecomms.cba.com.au/cv/4e28c09365e1c179bd89ab131200ec829530c314
2025-02-19 14:54:13.205 | INFO     | scraper:skip:130 - Skipping domain https://ecomms.cba.com.au/cv/4e28c09365e1c179bd89ab131200ec829530c314
2025-02-19 14:54:13.206 | INFO     | __main__:scrape:12 - Scraping 15: https://www.commbroker.com.au/Net/Documentum/credit-policy/acceptable-excluded-purposes.aspx
2025-02-19 14:54:13.709 | INFO     | scraper:save_content:56 - Downloading https://www.commbroker.com.au/Net/Documentum/credit-policy/acceptable-excluded-purposes.aspx to /home/kevinmcisaac/Projects/policy-pal-pages/policy/CBA/CommBroker//Net/Documentum/credit-policy/acceptable-excluded-purposes.aspx
2025-02-19 14:54:13.745 | INFO     | __main__:scrape:12 - Scraping 14: https://www.commbroker.com.au/Net/Doc

232


In [7]:
import glob
glob.glob("policy/CBA/CommBroker/**/*.\{aspx\}", recursive=True)

[]

In [11]:
import glob
import os

def find_html_aspx_files(directory, extentions:list):  # Default to current directory
    """Finds HTML and ASPX files in a directory and its subdirectories."""

    all_files = []

    for root, _, files in os.walk(directory):  # Walk through directory and subdirectories
        for file in files:
            if file.endswith(extentions): # Check file extensions
                all_files.append(os.path.join(root, file)) # Add full path

    return all_files

find_html_aspx_files("policy/CBA/CommBroker", extentions=(".html", ".htm", ".aspx"))

['policy/CBA/CommBroker/Net/Documentum/credit-policy/acceptable-excluded-purposes.aspx',
 'policy/CBA/CommBroker/Net/Documentum/credit-policy/alternateservicing.aspx',
 'policy/CBA/CommBroker/Net/Documentum/credit-policy/eligible-excluded-borrowers.aspx',
 'policy/CBA/CommBroker/Net/Documentum/credit-policy/debt-to-income-ratio.aspx',
 'policy/CBA/CommBroker/Net/Documentum/credit-policy/verifying-expenditure.aspx',
 'policy/CBA/CommBroker/Net/Documentum/credit-policy/foreign-currency-income.aspx',
 'policy/CBA/CommBroker/Net/Documentum/credit-policy/co-borrowers.aspx',
 'policy/CBA/CommBroker/Net/Documentum/credit-policy/eligible-borrowers.aspx',
 'policy/CBA/CommBroker/Net/Documentum/credit-policy/eligible-income.aspx',
 'policy/CBA/CommBroker/Net/Documentum/credit-policy/satisfactory-security.aspx',
 'policy/CBA/CommBroker/Net/Documentum/credit-policy/serviceability.aspx',
 'policy/CBA/CommBroker/Net/Documentum/credit-policy/calculatingproductcommitments.aspx',
 'policy/CBA/CommBroke